In [2]:
import pandas as pd
import json

Getting all stops from stops by line (the API method is off)

In [3]:
with open('BART_stops_by_line.json', 'r') as json_file:
    BART_stops = json.load(json_file)
BART_stops

[{'Id': 'Yellow-S',
  'Contents': {'ResponseTimestamp': '2024-03-04T23:52:56-08:00',
   'dataObjects': {'id': 'BA',
    'ScheduledStopPoint': [{'id': '12TH',
      'Extensions': {'LocationType': '0',
       'PlatformCode': None,
       'ParentStation': 'place_12TH',
       'ValidBetween': {'FromDate': '2024-01-15T00:00:00-08:00',
        'ToDate': '2024-08-11T23:59:00-08:00'}},
      'Name': '12th Street / Oakland City Center',
      'Location': {'Longitude': '-122.271589', 'Latitude': '37.803471'},
      'Url': None,
      'StopType': 'onstreetBus'},
     {'id': '16TH',
      'Extensions': {'LocationType': '0',
       'PlatformCode': None,
       'ParentStation': 'place_16TH',
       'ValidBetween': {'FromDate': '2024-01-15T00:00:00-08:00',
        'ToDate': '2024-08-11T23:59:00-08:00'}},
      'Name': '16th Street / Mission',
      'Location': {'Longitude': '-122.419755', 'Latitude': '37.765176'},
      'Url': None,
      'StopType': 'onstreetBus'},
     {'id': '19TH',
      'Extensi

In [11]:
# get list of dictionary of all stops with just: name, id, location
all_stops = [b['Contents']['dataObjects']['ScheduledStopPoint']  for b in BART_stops]
# flattens stops
all_stops = [stop for line in all_stops for stop in line]
# get necessary data
all_stops = [{'id':stop['id'],'Name':stop['Name'], 'Location':stop['Location']} for stop in all_stops]
# remove duplicates using id
unique_ids = []
unique_stops = []
for s in all_stops:
    if s['id'] not in unique_ids:
        unique_stops.append(s)
        unique_ids.append(s['id'])
print(len(unique_stops)) # reduced duplicate stops
len(all_stops)

50


234

In [12]:
file_path = 'BART_stops.json'
with open(file_path, 'w') as json_file:
    json.dump(unique_stops, json_file, indent=2)

Get min and max long & lat for leaflet map

In [4]:
with open('BART_stops.json', 'r') as json_file:
    BART_stops = json.load(json_file)
# BART_stops

[{'id': '12TH',
  'Name': '12th Street / Oakland City Center',
  'Location': {'Longitude': '-122.271589', 'Latitude': '37.803471'}},
 {'id': '16TH',
  'Name': '16th Street / Mission',
  'Location': {'Longitude': '-122.419755', 'Latitude': '37.765176'}},
 {'id': '19TH',
  'Name': '19th Street Oakland',
  'Location': {'Longitude': '-122.268722', 'Latitude': '37.80806'}},
 {'id': '24TH',
  'Name': '24th Street / Mission',
  'Location': {'Longitude': '-122.418518', 'Latitude': '37.752422'}},
 {'id': 'ANTC',
  'Name': 'Antioch',
  'Location': {'Longitude': '-121.780353', 'Latitude': '37.995415'}},
 {'id': 'BALB',
  'Name': 'Balboa Park',
  'Location': {'Longitude': '-122.44751', 'Latitude': '37.721758'}},
 {'id': 'CIVC',
  'Name': 'Civic Center / UN Plaza',
  'Location': {'Longitude': '-122.413879', 'Latitude': '37.77945'}},
 {'id': 'COLM',
  'Name': 'Colma',
  'Location': {'Longitude': '-122.466167', 'Latitude': '37.684642'}},
 {'id': 'CONC',
  'Name': 'Concord',
  'Location': {'Longitude'

In [6]:
locations = [s['Location'] for s in BART_stops]
latitudes = [l['Latitude'] for l in locations]
longitudes = [l['Longitude'] for l in locations]
print(min(latitudes),max(latitudes))
print(min(longitudes),max(longitudes))

37.368495 38.01895
-121.780353 -122.468994


Order the stops for each line --> Sam is doing this

### Fare scraping notes
https://www.bart.gov/tickets/calculator

Only patterns I found:
- minimum fare (w/o crossing the bay): $2.30
- crossing the bay: $3.75

Inconsistent patterns:
- crossing the bay + 1 stop, still: $3.75
- crossing the bay + 2 stops, sometimes: $3.75, sometimes: $4.05
- crossing the bay + 3 stops, still; $4.35
- ... seems to be $0.30 per extra stop? Nope.

In [2]:
# scrape fares for each combination of stops...
fare_url = 'https://www.bart.gov/tickets/calculator';
fare_table_url = 'https://www.transit.wiki/BART_fares';

Convert bart fares from csv to json

In [7]:
BART_fares = pd.read_csv('BART_fares.csv', index_col=0)
BART_fares

,12th Street/ Oakland City Center,16th Street/ Mission,19th Street Oakland,24th Street/ Mission,Antioch,Ashby,Balboa Park,Bay Fair,Berryessa/ North San Jose,Castro Valley,...,San Bruno,San Francisco International Airport,San Leandro,South Hayward,South San Francisco,Union City,Walnut Creek,Warm Springs/ South Fremont,West Dublin/ Pleasanton,West Oakland
Station,,,,,,,,,,,,,,,,,,,,,
Civic Center/UN Plaza,4.05,2.30,4.05,2.30,8.60,4.55,2.30,5.60,8.85,5.90,...,5.05,10.55,5.30,6.25,4.60,6.85,6.20,7.75,7.00,3.75
Embarcadero,4.05,2.30,4.05,2.30,8.60,4.55,2.30,5.60,8.85,5.90,...,5.05,10.55,5.30,6.25,4.60,6.85,6.20,7.75,7.00,3.75
Montgomery Street,4.05,2.30,4.05,2.30,8.60,4.55,2.30,5.60,8.85,5.90,...,5.05,10.55,5.30,6.25,4.60,6.85,6.20,7.75,7.00,3.75
Powell Street,4.05,2.30,4.05,2.30,8.60,4.55,2.30,5.60,8.85,5.90,...,5.05,10.55,5.30,6.25,4.60,6.85,6.20,7.75,7.00,3.75
24th Street/Mission,4.40,2.30,4.40,6.75,8.75,4.80,2.30,5.75,9.00,6.05,...,4.75,10.35,5.45,6.40,4.30,7.00,6.35,7.90,7.15,4.15
Glen Park,4.65,2.30,4.65,2.30,8.95,5.00,2.30,5.90,9.20,6.25,...,4.45,10.05,5.65,6.60,4.00,7.15,6.50,8.05,7.35,4.45
Balboa Park,4.80,2.30,4.80,2.30,9.00,5.10,6.75,6.00,9.25,6.30,...,4.30,9.90,5.70,6.65,3.80,7.25,6.60,8.15,7.40,4.65
Daly City,4.95,3.60,4.95,3.60,9.15,5.20,3.60,6.15,9.40,6.45,...,3.80,9.65,5.85,6.80,3.80,7.40,6.75,8.30,7.55,4.90
West Oakland,2.30,4.05,2.30,4.15,6.70,2.30,4.65,3.55,6.95,4.00,...,5.35,10.80,3.10,4.35,5.05,4.90,4.25,5.80,5.10,6.75


In [6]:
BART_fares_json = BART_fares.to_json('BART_fares.json', orient='index') 

### Clean ordered stop data

In [11]:
ordered = pd.read_csv('BART_stops_by_line_ordered.csv')
ordered.head()

,Id,Contents__ResponseTimestamp,Contents__dataObjects__id,Contents__dataObjects__ScheduledStopPoint__id,Contents__dataObjects__ScheduledStopPoint__Extensions__LocationType,Contents__dataObjects__ScheduledStopPoint__Extensions__PlatformCode,Contents__dataObjects__ScheduledStopPoint__Extensions__ParentStation,Contents__dataObjects__ScheduledStopPoint__Extensions__ValidBetween__FromDate,Contents__dataObjects__ScheduledStopPoint__Extensions__ValidBetween__ToDate,Contents__dataObjects__ScheduledStopPoint__Name,Contents__dataObjects__ScheduledStopPoint__Location__Longitude,Contents__dataObjects__ScheduledStopPoint__Location__Latitude,Contents__dataObjects__ScheduledStopPoint__Url,Contents__dataObjects__ScheduledStopPoint__StopType
0,Yellow-S,2024-03-04T23:52:56-08:00,BA,ANTC,0,NaN,place_ANTC,2024-01-15T00:00:00-08:00,2024-08-11T23:59:00-08:00,Antioch,-121.780353,37.995415,NaN,onstreetBus
1,NaN,NaN,NaN,PCTR,0,NaN,place_PCTR,2024-01-15T00:00:00-08:00,2024-08-11T23:59:00-08:00,Pittsburg Center,-121.889011,38.016873,NaN,onstreetBus
2,NaN,NaN,NaN,PITT,0,NaN,place_PITT,2024-01-15T00:00:00-08:00,2024-08-11T23:59:00-08:00,Pittsburg / Bay Point,-121.944248,38.018950,NaN,onstreetBus
3,NaN,NaN,NaN,NCON,0,NaN,place_NCON,2024-01-15T00:00:00-08:00,2024-08-11T23:59:00-08:00,North Concord / Martinez,-122.024561,38.003407,NaN,onstreetBus
4,NaN,NaN,NaN,CONC,0,NaN,place_CONC,2024-01-15T00:00:00-08:00,2024-08-11T23:59:00-08:00,Concord,-122.029126,37.973773,NaN,onstreetBus


In [12]:
# remove unnecessary columns
ordered = ordered.loc[:, [
    'Id',
    'Contents__dataObjects__ScheduledStopPoint__id',
    # 'Contents__dataObjects__ScheduledStopPoint__Name',
    # 'Contents__dataObjects__ScheduledStopPoint__Location__Longitude',
    # 'Contents__dataObjects__ScheduledStopPoint__Location__Latitude'
]]
# and rename columns
ordered.columns = ['line_id',
                   'stop_id',
                #    'stop_name',
                #    'longitude',
                #    'latitude'
                   ]
ordered.head()

,line_id,stop_id
0,Yellow-S,ANTC
1,NaN,PCTR
2,NaN,PITT
3,NaN,NCON
4,NaN,CONC


In [13]:
# populate line_id down to fill NaN
ordered['line_id'] = ordered['line_id'].fillna(method='ffill')
print(len(ordered.index))
# only need to keep 1 of north or south, remove South:
ordered = ordered[~ordered['line_id'].str.contains('-S')]
print(len(ordered.index))
ordered.head()

230
115


,line_id,stop_id
28,Orange-N,BERY
29,Orange-N,MLPT
30,Orange-N,WARM
31,Orange-N,FRMT
32,Orange-N,UCTY


In [14]:
# convert to and export as json in records orientation
ordered.to_json('BART_stops_by_line_ordered.json', orient='records')

In [21]:
# also save data as object orientation:
# {line_name: [stop1, stop2, ..., stopN]}
grouped_ordered = ordered.groupby('line_id')['stop_id'].apply(list).reset_index()
grouped_ordered
result_dict = grouped_ordered.set_index('line_id')['stop_id'].to_dict()
with open('BART_lines_ordered_stops.json', 'w') as json_file:
    json.dump(result_dict, json_file, indent=4)
# grouped_ordered.to_json('BART_line_stops_ordered.json', orient='split')